In [1]:
%run '00_database_connectivity_setup.ipynb'

C:\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [145]:
%%execsql
CREATE OR REPLACE FUNCTION nate.explode_bom(material text, plant text) RETURNS text AS $$

DECLARE
        explosion_complete bool := false;
        level int2 := 0;
        tmpcount int;
    
BEGIN
                                                                                                                                                                       
         TRUNCATE TABLE nate.exploded_table;                                                                                                                                       
         INSERT INTO nate.exploded_table                                                                                                                                              
                                                                                                                                                                                  
         (                                                                                                                                                                                     
             SELECT   0 as lvl, component,                                                                                                                                                     
             CASE WHEN un_1='EA' or UN_1='PCE' then (1.0 * translate(quantity,',','')::numeric(20,4)) ELSE 1.0 END as quantity,                                                                
             un_1 as unit                                                                                                                                                                      
             FROM data_science.material_bom_new a                                                                                                                                                    
             WHERE a.material=$1 and a.plnt=$2                                                                                                                                   
         );                                                                                                                                                     
            
        ANALYZE nate.exploded_table;
        
        WHILE (NOT explosion_complete) LOOP
            
            level := level + 1;
        
           /* TRUNCATE TABLE nate.exploded_temp;  
            
            INSERT INTO nate.exploded_temp(                                                                                                                                                                  
                     SELECT                                                                                                                                                                    
                        level as lvl, t.Component,                                                                                                                                             
                        CASE WHEN t.un_1='EA' or t.un_1='PCE' then (1.0 * 
                        translate(t.quantity,',','')::numeric(20,4) * 
                        translate(s.quantity,',','')::numeric(20,4)) 
                        ELSE 1.0 END as quantity, t.un_1 as unit                                                                                                                                                         
                     FROM data_science.material_bom_new t                                                                                                                                          
                     INNER JOIN nate.exploded_table s  ON t.material = s.component                                                                                                          
                     WHERE t.plnt=$2 and s.lvl=level-1                                                                                                                                  
                 ); 
                 SELECT COUNT(*) FROM nate.exploded_temp INTO tmpcount; */
                
            tmpcount := (SELECT COUNT(*)  FROM data_science.material_bom_new t                                                                                                                                          
                     INNER JOIN nate.exploded_table s  ON t.material = s.component                                                                                                          
                     WHERE t.plnt=$2 and s.lvl=level-1 );
            IF ((tmpcount = 0) OR level > 15) THEN explosion_complete = true; 
            ELSE 
                 INSERT INTO nate.exploded_temp(                                                                                                                                                                  
                     SELECT                                                                                                                                                                    
                        level as lvl, t.Component,                                                                                                                                             
                        CASE WHEN t.un_1='EA' or t.un_1='PCE' then (1.0 * 
                        translate(t.quantity,',','')::numeric(20,4) * 
                        translate(s.quantity,',','')::numeric(20,4)) 
                        ELSE 1.0 END as quantity, t.un_1 as unit                                                                                                                                                         
                     FROM data_science.material_bom_new t                                                                                                                                          
                     INNER JOIN nate.exploded_table s  ON t.material = s.component                                                                                                          
                     WHERE t.plnt=$2 and s.lvl=level-1                                                                                                                                  
                 ); 
            END IF;
            
        END LOOP;
        
        RETURN ('Successfully exploded ' || $1 || ' to ' || level || ' levels');

END;
$$ LANGUAGE plpgsql

In [65]:
%%showsql
SELECT nate.explode_bom('P1000014514','EWHG')

,explode_bom
0,Successfully exploded P1000014514 to 7 levels


In [66]:
%%showsql
SELECT nate.explode_bom('P1000014514','HCS0')

,explode_bom
0,Successfully exploded P1000014514 to 1 levels


In [61]:
%%showsql
SELECT COUNT(*) FROM nate.exploded_table

,count
0,0


In [17]:
%%showsql
SELECT material,plnt FROM data_science.material_bom_new limit 100

,material,plnt
0,P1000032576,
1,SP-83-16979,SFBN
2,SP-83-16979,SFBN
3,SP-83-16979,SFBN
4,P7000034954,
5,7500206,KOS1
6,7500206,KOS1
7,7500206,KOS1
8,7500206,KOS1
9,7500088,KOS1


In [62]:
%%showsql
select smithd2.explode_bom_new('P173716','HCS0')

,explode_bom_new
0,Successfully exploded P173716 to 1 levels


In [17]:
%%showsql
select smithd2.explode_bom_new('P1000014514','EWHG')

,explode_bom_new
0,Successfully exploded P1000014514 to 7 levels


In [74]:
%%showsql
select count(*) from smithd2.exploded_table_new 
LIMIT 1000;

,count
0,1517


THE FUNCTION FINALLY WORKS!

In [4]:
%%showsql
SELECT COUNT(DISTINCT material), COUNT(DISTINCT component) FROM nate.simple_features

,count,count
0,4583,16443


In [14]:
%%showsql
SELECT COUNT(DISTINCT material) FROM nate.simple_features WHERE plant='EWHG'

,count
0,489


Why so low???

In [3]:
%%showsql
SELECT * FROM data_science.ordercomponents_int limit 1

,ordr,oper,bomitem,reservationno,resvitem,reqtyp,deleted,material,plant,sloc,qtyreqd,qtywithdrawn,uom,valuewithdrawn,crcy
0,1238409,0010,20,52525127,2,AR,1,P1000091011,KOS2,0001,2.0,2.0,EA,2466.17,NOK


In [10]:
%%showsql
    SELECT DISTINCT material FROM nate.simple_features WHERE plant = 'EWHG'
    EXCEPT
    SELECT material FROM data_science.ordercomponents_int WHERE plant = 'EWHG'

DatabaseError: Execution failed on sql '    SELECT material FROM nate.simple_features WHERE plnt = 'EWHG'
    EXCEPT
    SELECT material FROM data_science.ordercomponents_int': column "plnt" does not exist
LINE 1: ...  SELECT material FROM nate.simple_features WHERE plnt = 'EW...
                                                             ^


In [8]:
%%showsql
    SELECT component FROM nate.simple_features
    EXCEPT
    SELECT material FROM data_science.ordercomponents_int

,component
0,701-073-249
1,701-073-249
2,701-073-249
3,P114561-0000
4,P114561-0000
5,P114561-0000
6,P114561-0000
7,P114561-0000
8,P114561-0000
9,P114561-0000


In [18]:
%%showsql
SELECT COUNT(DISTINCT a.cs_order) 
FROM sap_bi.bic_azcs_o0100_int a 
JOIN sap_bi.bic_azequip00_int b 
ON a.equipment = b.equipment

,count
0,315571


In [19]:
%%showsql
SELECT COUNT(DISTINCT b.material) 
FROM sap_bi.bic_azcs_o0100_int a 
JOIN sap_bi.bic_azequip00_int b 
ON a.equipment = b.equipment

,count
0,18800


In [20]:
%%showsql 
SELECT COUNT(DISTINCT cs_order) FROM sap_bi.bic_azcs_o0100_int

,count
0,505625


In [16]:
%%showsql 
SELECT COUNT(DISTINCT ordr) FROM data_science.ordercomponents_int

,count
0,273802


In [17]:
%%showsql
SELECT COUNT(DISTINCT ordr) FROM nate.simple_features

,count
0,26498


In [22]:
%%showsql
SELECT count(DISTINCT b.material)
FROM sap_bi.bic_azcs_o0100_int a 
JOIN sap_bi.bic_azequip00_int b 
ON a.equipment = b.equipment
WHERE a.planplant='EWHG' OR a.maintplant='EWHG'

,count
0,4512


In [72]:
%%showsql
SELECT COUNT(DISTINCT c.material) FROM
(SELECT DISTINCT b.material
FROM sap_bi.bic_azcs_o0100_int a 
JOIN sap_bi.bic_azequip00_int b 
ON a.equipment = b.equipment
WHERE a.planplant='EWHG' OR a.maintplant='EWHG' OR a.planplant='HCS0' OR a.maintplant='HCS0'
UNION ALL
(SELECT DISTINCT material FROM nate.simple_features WHERE plant='EWHG')) c

,count
0,4872


In [69]:
%%showsql
SELECT COUNT(DISTINCT c.material) FROM
(SELECT DISTINCT b.material
FROM sap_bi.bic_azcs_o0100_int a 
JOIN sap_bi.bic_azequip00_int b 
ON a.equipment = b.equipment
WHERE a.planplant='HCS0' OR a.maintplant='HCS0'
UNION ALL
(SELECT DISTINCT material FROM nate.simple_features WHERE plant='HCS0')) c

,count
0,4256


Okay, so almost all of the materials that were present in simple_features also survive the 
join with the equipment table - I'll exclude the 45 that don't for now.

In [55]:
%%showsql
SELECT DISTINCT b.material, a.maintplant, a.planplant
FROM sap_bi.bic_azcs_o0100_int a 
JOIN sap_bi.bic_azequip00_int b 
ON a.equipment = b.equipment
WHERE (a.planplant='EWHG' OR a.maintplant='EWHG') AND (b.material)<>'NON FMC PART'

,material,maintplant,planplant
0,000000000007101596,EWHG,HCS0
1,P1000126580,EWHG,HCS0
2,P126771-0014,EWHG,EWHG
3,000000000007103891,EWHG,HCS0
4,18-318-311,EWHG,EWHG
5,P133954-0021,EWHG,HCS0
6,P1000090124,EWHG,HCS0
7,P170135,EWHG,HCS0
8,P123276-0021,,EWHG
9,CS-000331,EWHG,EWHG


In [ ]:
%%showsql
SELECT nate.explode_bom('P174812','EWHG')

In [57]:
%%showsql
SELECT * FROM nate.exploded_table

,lvl,component,quantity,unit
0,4,P1000005774,1.0,EA
1,1,P1000013432,1.0,IN
2,2,P1000033721,1.0,EA
3,1,P170565,1.0,EA
4,1,P1000041906,1.0,EA
5,0,P1000041883,1.0,EA
6,0,P1000041905,1.0,EA
7,3,P1000033719,1.0,EA


In [7]:
%%execsql
CREATE TABLE nate.ewhg_exploded_bom (material text, lvl int2, component text, quantity numeric, unit text)

In [120]:
%%execsql
CREATE OR REPLACE FUNCTION nate.createthebom() RETURNS void AS $$

DECLARE
    mat text;
BEGIN
    FOR mat IN
        (SELECT DISTINCT b.material
        FROM sap_bi.bic_azcs_o0100_int a 
        JOIN sap_bi.bic_azequip00_int b 
        ON a.equipment = b.equipment
        WHERE (a.planplant='EWHG' OR a.maintplant='EWHG') AND (b.material)<>'NON FMC PART')
    LOOP
        PERFORM nate.explode_bom(mat,'EWHG');
        INSERT INTO nate.ewhg_exploded_bom (SELECT * FROM (SELECT mat AS matrl) a 
                                            JOIN (SELECT lvl FROM nate.exploded_table) b ON true=true);
    END LOOP;
END;

$$ LANGUAGE plpgsql


In [90]:
%%execsql
SELECT madlib.summary('data_science.ordercomponents_int','nate.ordercompsum')

DatabaseError: Execution failed on sql 'SELECT madlib.summary('data_science.ordercomponents_int','nate.ordercompsum')': plpy.Error: Summary error: Output table 'nate.ordercompsum' already exists. (plpython.c:4648)
DETAIL:  Drop it before calling the function.
CONTEXT:  Traceback (most recent call last):
  PL/Python function "summary", line 22, in <module>
    get_distinct, get_quartiles, ntile_array, how_many_mfv, get_estimates)
  PL/Python function "summary", line 77, in summary
  PL/Python function "summary", line 353, in run
  PL/Python function "summary", line 97, in _validate_paras
  PL/Python function "summary", line 69, in _validate_table_names
  PL/Python function "summary", line 569, in output_tbl_valid
PL/Python function "summary"


In [6]:
%%showsql
SELECT * from nate.ordercompsum

,group_by,group_by_value,target_column,column_number,data_type,row_count,distinct_values,missing_values,blank_values,fraction_missing,fraction_blank,mean,variance,min,max,first_quartile,median,third_quartile,most_frequent_values,mfv_frequencies
0,None,None,ordr,1,varchar,3314620,282676,0,0.0,0.0,0.000000,NaN,NaN,6.00,12.00,NaN,NaN,NaN,"[301066913, 301549156, 300990325, 300926994, 301539165, 300942648, 301235185, 301524685, 301063549, 301266554]","[4101, 3839, 3768, 3753, 3731, 3456, 3451, 3346, 3343, 3340]"
1,None,None,oper,2,bpchar,3314620,1950,0,0.0,0.0,0.000000,NaN,NaN,4.00,4.00,NaN,NaN,NaN,"[0010, 0010, 0001, 0001, 0100, 0100, 0030, 0030, 0005, 0005]","[1464090, 1464090, 707245, 707245, 304047, 304047, 229673, 229673, 88701, 88701]"
2,None,None,bomitem,3,text,3314620,3130,0,968.0,0.0,0.000292,NaN,NaN,0.00,4.00,NaN,NaN,NaN,"[1, 1, 2, 2, 3, 3, 10, 10, 4, 4]","[404936, 404936, 292046, 292046, 230495, 230495, 182802, 182802, 160951, 160951]"
3,None,None,reservationno,4,text,3314620,297599,0,0.0,0.0,0.000000,NaN,NaN,7.00,8.00,NaN,NaN,NaN,"[49852429, 52272573, 79722898, 80696960, 81157113, 79723063, 79721703, 52179930, 52179929, 79722244]","[3714, 3516, 3516, 3423, 3392, 3385, 3350, 3274, 3272, 3259]"
4,None,None,resvitem,5,text,3314620,2453,0,0.0,0.0,0.000000,NaN,NaN,1.00,4.00,NaN,NaN,NaN,"[1, 1, 2, 2, 3, 3, 4, 4, 5, 5]","[257881, 257881, 177651, 177651, 138851, 138851, 111385, 111385, 94975, 94975]"
5,None,None,reqtyp,6,bpchar,3314620,2,0,0.0,0.0,0.000000,NaN,NaN,2.00,2.00,NaN,NaN,NaN,"[AR, AR, BB, BB]","[3314614, 3314614, 6, 6]"
6,None,None,deleted,7,varbit,3314620,2,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1, 1, 0, 0]","[3170019, 3170019, 144601, 144601]"
7,None,None,material,8,varchar,3314620,115990,0,15723.0,0.0,0.004744,NaN,NaN,0.00,18.00,NaN,NaN,NaN,"[, 200002801, 200002801, 200000496, 200000496, P6000025422, P6000025422, 100030402, 100030402, 100021358]","[17577, 17443, 17443, 14755, 14755, 14049, 14049, 14020, 14020, 13568]"
8,None,None,plant,9,bpchar,3314620,29,0,0.0,0.0,0.000000,NaN,NaN,4.00,4.00,NaN,NaN,NaN,"[KOS1, KOS1, EWHG, EWHG, 4024, 4024, EWSC, EWSC, 4011, GSEA]","[717492, 717492, 561857, 561857, 469206, 469206, 339783, 339783, 269797, 212253]"
9,None,None,sloc,10,bpchar,3314620,237,0,103991.0,0.0,0.031373,NaN,NaN,0.00,4.00,NaN,NaN,NaN,"[0001, 0001, 0005, 0005, HRDY, HRDY, , , 0002, 0002]","[1794392, 1794392, 974547, 974547, 134511, 134511, 103991, 103991, 95401, 95401]"


In [ ]:
%%showsql
SELECT nate.createthebom()

In [3]:
%%execsql
DROP TABLE IF EXISTS nate.exploded_temp;
CREATE TABLE nate.exploded_temp (lvl int2, component text, quantity numeric, unit text)

In [4]:
%%execsql
DROP TABLE IF EXISTS nate.exploded_table;
CREATE TABLE nate.exploded_table (lvl int2, component text, quantity numeric, unit text)

In [138]:
%%showsql
SELECT oid from pg_class where relname = 'exploded_table' or relname = 'exploded_temp'

,oid
0,221262
1,954070
2,955431
3,963159


In [139]:
%%showsql
SELECT oid from pg_class where relname = 'exploded_table' or relname = 'exploded_temp'

,oid
0,221262
1,954070
2,955431
3,963159


In [146]:
%%showsql
SELECT pg_database_size(fmc_poc)

DatabaseError: Execution failed on sql: SELECT pg_database_size(fmc_poc)
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

unable to rollback

In [ ]:
%%showsql
select * from nate.ewhg_exploded_bom limit 10